# Faster R-CNN Training using Tensorflow
___

### Before started!
- navigate to `Runtime` menu, and choose `change runtime type`,
- then, change `hardware acceleration` to `GPU`
- click `connect` button in top-right colab

# 1. Install Library
### 1.A. Install Tensorflow 1.15 & OpenCV 4.4.0

In [ ]:
!pip install tensorflow_gpu==1.15

In [ ]:
!pip install opencv-python==4.4.0.46
!pip install opencv-contrib-python==4.4.0.46

### 1.B. Install Other Library & Clone Tensorflow Model

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython tf_slim
!pip install -q pycocotools
!pip install -q Cython contextlib2 pillow lxml matplotlib

!git clone https://github.com/tensorflow/models.git

### 1.C. Build Tensorflow Model Builder

In [ ]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

import os
os.environ['PYTHONPATH'] += ":/content/models"

import sys
sys.path.append("/content/models")
!python object_detection/builders/model_builder_test.py

# 2. Prepare Dataset
### 2.A Upload and extract dataset.
- upload `dataset.zip` using this script,
- `dataset.zip` is created by running `dataset_builder.ipynb` from this repository ([ObjectDetection-Tensorflow](https://github.com/Muhammad-Yunus/ObjectDetection-Tensorflow)) in your local computer.

In [ ]:
%cd /content

import shutil
from zipfile import ZipFile
from google.colab import files
from os import path

print("Upload `dataset.zip` to colab :")
uploaded = files.upload()
  
for fileName, data in uploaded.items():
  with open('dataset.zip', 'wb') as f:
    f.write(data)
    f.close()
    print('saved dataset (.zip) file ' + fileName)

ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

### 2.B. Prepare Dataset 
- Convert XML PASCAL VOC to CSV
- Create TF_Record from generated CSV

In [6]:
%cd /content/dataset

!python xml_to_csv.py

/content/dataset
Successfully converted xml to csv.
Successfully converted xml to csv.


- edit `class_text_to_int()` function in `generated_tfrecord.py` with label and index label we have.

In [ ]:
%cd /content/dataset

!python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=data/train.record --image_dir=images/train

!python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=data/test.record --image_dir=images/test

# 3. Download pretrained model

- Pretrained model (Tensorflow 1 Model Zoo) : [[tensorflow github](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md)]


In [8]:
%cd /content/dataset
%rm -rf models

%mkdir models
%mkdir models/inference_graph

/content/dataset


In [9]:
%cd /content/dataset/models

import os
import shutil
import glob
import urllib.request
import tarfile

MODEL = 'faster_rcnn_inception_v2_coco_2018_01_28'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'faster_rcnn_inception_v2'

with urllib.request.urlopen(DOWNLOAD_BASE+MODEL_FILE) as response, open(MODEL_FILE, 'wb') as out_file:
  shutil.copyfileobj(response, out_file)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/dataset/models


# 4. Copy & Edit Config File
- copy config model to `data/` folder

In [10]:
CONFIG_NAME = "faster_rcnn_inception_v2_coco.config"
shutil.copy(
    "/content/models/research/object_detection/samples/configs/" + CONFIG_NAME, 
    "/content/dataset/data/")

'/content/dataset/data/faster_rcnn_inception_v2_coco.config'

- open & edit config file in `/content/dataset/data/faster_rcnn_inception_v2_coco.config`, 

- navigate to line 10, change  `model`>`faster_rcnn`>`num_classes` to number of class on uploaded dataset (e.g/ 2).
- navigate to line 107,
```
fine_tune_checkpoint: "/Path to the pre-trained models/model.ckpt"
```
- change to 
```
/content/dataset/models/faster_rcnn_inception_v2/model.ckpt
```
- navigate to line 113, change `train_config`>`num_steps` to number of training step (e.g/ 2000)
- navigate to line 120,
```
train_input_reader: {
  tf_record_input_reader {
  input_path: "/Path to the tfrecord/train.record"
  }
  label_map_path: "/Path to label map/object-detection.pbtxt"
}
```
- change `input_path` to,
```
/content/dataset/data/train.record
```
- change `label_map_path` to,
```
/content/dataset/data/object-detection.pbtxt
```

- navigate to line 134,
```
eval_input_reader: {
tf_record_input_reader {
input_path: "/Path to the tfrecord/test.record"
}
label_map_path: "/Path to label map/object-detection.pbtxt"
}
```
- change `input_path` to,
```
/content/dataset/data/test.record
```
- change `label_map_path` to,
```
/content/dataset/data/object-detection.pbtxt
```

# 5. Train model
Make and set train directory num_train_steps and num_eval_steps values to change train and eval steps in training process.



In [11]:
!cp /content/models/research/object_detection/legacy/train.py /content/models/research/object_detection

In [ ]:
%cd /content/models/research/object_detection/
%rm -rf training/*

!python train.py \
  --logtostderr \
  --train_dir=training/ \
  --pipeline_config_path=/content/dataset/data/faster_rcnn_inception_v2_coco.config

# 5. Tensorboard Training Visualisation + Ngrok
- **TensorBoard** is a tool for providing the `measurements` and `visualizations` needed during the machine learning workflow. It enables `tracking` experiment `metrics` like `loss` and `accuracy`, visualizing the model graph, projecting embeddings to a lower dimensional space, and much more.
- **ngrok** is secure `introspectable tunnels` to `localhost` webhook development tool and debugging tool.

In [ ]:
%cd /content/models/research/object_detection/training/
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [18]:
LOG_DIR = '/content/models/research/object_detection/training/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [19]:
get_ipython().system_raw('./ngrok http 6006 &')

In [20]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://b4f9d7cbe24c.ngrok.io


- close ngrok - tensorboard for training log visualization

In [21]:
!ps aux | grep ngrok

root        4478  0.8  0.2 122768 29960 ?        Sl   00:53   0:00 ./ngrok http 6006
root        4534  0.0  0.0  39204  6436 ?        S    00:54   0:00 /bin/bash -c ps aux | grep ngrok
root        4536  0.0  0.0  38576  5596 ?        S    00:54   0:00 grep ngrok


In [22]:
!kill -9 4478  #<---ngrok tensorboard train pid--->

#. 6 Tensor Board Eval Visualization

In [23]:
%cd /content/models/research/object_detection/
!cp /content/models/research/object_detection/legacy/eval.py /content/models/research/object_detection

/content/models/research/object_detection


In [ ]:
!pip install lvis

- run evaluation metric (`eval.py`) to find **mAP** of object detection model,

In [ ]:
%rm -rf eval/*

!python eval.py \
  --logtostderr \
  --pipeline_config_path=/content/dataset/data/faster_rcnn_inception_v2_coco.config \
  --checkpoint_dir=training/ \
  --eval_dir=eval/

In [ ]:
%cd /content/models/research/object_detection/training/
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [27]:
LOG_DIR = '/content/models/research/object_detection/eval/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6007 &'
    .format(LOG_DIR)
)

In [28]:
get_ipython().system_raw('./ngrok http 6007 &')

In [29]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://a5e52255d1e4.ngrok.io


# 7. Export trained model

Export trained model with highest step number in filename.
- change `model.ckpt-xxxxx` with `xxxxx` is number of step in trainingphase.

In [ ]:
%cd /content/models/research/object_detection
%rm -rf inference_graph/*

!python export_inference_graph.py \
  --input_type image_tensor \
  --pipeline_config_path /content/dataset/data/faster_rcnn_inception_v2_coco.config \
  --trained_checkpoint_prefix /content/models/research/object_detection/training/model.ckpt-120000 \
  --output_directory /content/dataset/models/inference_graph

# 8. Generate OpenCV DNN Config for Inference
- generate `faster_rcnn_inception_v2_custom_dataset.pbtxt` for OpenCV DNN inferencing

In [ ]:
%cd /content
!python dataset/cvdnn_config_generator/tf_text_graph_faster_rcnn.py \
    --input dataset/models/inference_graph/frozen_inference_graph.pb \
    --output dataset/models/inference_graph/faster_rcnn_inception_v2_custom_dataset.pbtxt \
    --config dataset/models/inference_graph/pipeline.config

# 9. Test Model (Inferencing)

- upload arial font (`arial.ttf`) from cloned repo ([ObjectDetection-Tensorflow](https://github.com/Muhammad-Yunus/ObjectDetection-Tensorflow)) using script below,

In [ ]:
uploaded = files.upload()
  
for name, data in uploaded.items():
  with open('/content/models/research/object_detection/utils/arial.ttf', 'wb') as f:
    f.write(data)
    f.close()
    print('saved file ' + name)

- open `/content/models/research/object_detection/utils/visualization_utils.py` and navigate to line `212`, change *font size* to a proper value (e.g/ 50).
    ```
    try:
      font = ImageFont.truetype('arial.ttf', 50)
    except IOError:
      font = ImageFont.load_default()
    ```

- Upload image test

In [ ]:
%cd /content
# upload image from local computer to colab

from google.colab import files
from os import path

uploaded = files.upload()
  
for name, data in uploaded.items():
  with open('image1.jpg', 'wb') as f:
    f.write(data)
    f.close()
    print('saved file ' + name)

- predict on `image1.jpg`

In [ ]:
%cd /content/models/research/object_detection

# [NOTE!] this sample implementation using Tensorflow ver 1.14

import os
import sys
import numpy as np
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

from utils import label_map_util
from object_detection.utils import ops as utils_ops
from utils import visualization_utils as vis_util
import importlib
importlib.reload(vis_util)

# This is needed to display the images.
%matplotlib inline

# What model to download.
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/content/dataset/models/inference_graph' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/content/dataset/data', 'object-detection.pbtxt')

NUM_CLASSES = 2 #[TODO!] change to number of training class

TEST_IMAGE_PATHS = [ os.path.join('/content/', 'image{}.jpg'.format(i)) for i in range(1, 2) ]

IMAGE_SIZE = (8, 17)

# load frozen inference graph
frcnn_graph = tf.Graph()
with frcnn_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
# load label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


# PIL image to numpy array converter
def PIL_to_numpy(image):
  (w, h) = image.size

  return np.array(image.getdata()).reshape((h, w, 3)).astype(np.uint8)

def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
          
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
                                      'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict


# -------------- main block --------------
for image_path in TEST_IMAGE_PATHS:
  # read image using PIL
  image = Image.open(image_path)
  image_np = PIL_to_numpy(image)

  # predict image
  output_dict = run_inference_for_single_image(image_np, frcnn_graph)

  # draw bousing box using `vis_util`
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  
  # show image
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)
  

# 10. Zip  & Download Inference Graph

In [ ]:
%cd /content/dataset/models
!zip -r inference_graph.zip inference_graph/frozen_inference_graph.pb inference_graph/faster_rcnn_inception_v2_custom_dataset.pbtxt

from google.colab import files
files.download("inference_graph.zip")

___
# Source :
- https://muchamadsyaiffudin.medium.com/object-detection-with-custom-dataset-faster-rcnn-on-google-colab-33b373a625eb
- https://colab.research.google.com/drive/1skc-BJ6HzuaIRbi3diP86lRnCTcNCond?usp=sharing